In [2]:
import csv
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from linkedin_api import Linkedin
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import sqlite3



In [5]:
keywordsRelatedToSponsorship = [None, "sponsorship", "visa", "right to work", "right to stay", "h1b", "h-1b", "authorized to work", "work authorization", "sponsor", "sponsored"]

In [4]:
from http.client import RemoteDisconnected
import random
import time


api = Linkedin('georgerithika7@gmail.com', 'RandomPassword2020!')

def getJobsFromLinkedIn(searchKeyword):
    # Fetch job search results
    print("Fetching Job")
    max_retries = 5
    for attempt in range(max_retries):
        print(attempt)
        try:
            with tqdm(total=1, desc="Retrieving Jobs", unit="job") as pbar:
                response = []
                if(searchKeyword is None): response = api.search_jobs()
                else: response = api.search_jobs(searchKeyword)
                pbar.update(1)
            print("Retrieved Jobs")

            # Extract job IDs from 'trackingUrn'
            job_ids = [
                job.get('trackingUrn', '').split(':')[-1]
                for job in response
                if job.get('trackingUrn', '').startswith('urn:li:jobPosting:')
            ]
            print("Mapped to IDs")
            
            with tqdm(total=len(job_ids), desc="Fetching Job Details", unit="job") as pbar:
                # Use ThreadPoolExecutor to fetch job details in parallel
                with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust 'max_workers' based on performance
                    # Define a wrapper function to update progress
                    def fetch_and_update(job_id):
                        job_detail = getJobDetailFromLinkedIn(job_id)
                        pbar.update(1)  # Update progress bar after each job detail is fetched
                        return job_detail

                    job_details_list = list(executor.map(fetch_and_update, job_ids))
        
            print("Fetching of Jobs Completed")
            return job_details_list
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError, RemoteDisconnected) as e:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt + random.uniform(0, 1)  # Exponential backoff with jitter
                print(f"Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
            else:
                print(f"Failed to fetch joblist after {max_retries} attempts.")
                return None

    
    
# Define a function to fetch job details for a given job ID
def getJobDetailFromLinkedIn(job_id):

    max_retries = 5
    for attempt in range(max_retries):
        try:

            job_data = api.get_job(job_id)

            # Only return specific fields that you need (e.g., title, company, location)
            filtered_job_data = {
                'job_posting_id': job_data.get('jobPostingId', 'N/A'),
                'title': job_data.get('title', 'N/A'),
                'company': job_data.get('companyDetails', {}).get('com.linkedin.voyager.deco.jobs.web.shared.WebCompactJobPostingCompany', {}).get('companyResolutionResult', {}).get('name', 'N/A'),
                'location': job_data.get('formattedLocation', 'N/A'),
                'description': job_data.get('description', {}).get('text', 'N/A'),
                'job_site': "LinkedIn"
            }
            return filtered_job_data
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError, RemoteDisconnected) as e:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt + random.uniform(0, 1)  # Exponential backoff with jitter
                print(f"Retrying in {wait_time:.2f} seconds...")
                time.sleep(wait_time)
            else:
                print(f"Failed to fetch details for job {job_id} after {max_retries} attempts.")
                return None


def getJobsForLinkedInAndSaveToCsvFile(keyword):
    linkedInData = getJobsFromLinkedIn(None)
    df = pd.DataFrame(linkedInData)
    print("Moving to csv file")

    with tqdm(total=len(linkedInData), desc="Moving to csv file", unit="job"):
        df.to_csv("linkedin_data.csv", mode="a", header=(keyword is None), index=False)


In [5]:
with ThreadPoolExecutor() as executor:
    for keyword in keywordsRelatedToSponsorship[7:]:
        future = executor.submit(getJobsForLinkedInAndSaveToCsvFile, keyword)
        future.result()
    print("Completed exporting Data from LinkedIn")

authorized to work
work authorization
sponsor
sponsored
Completed exporting Data from LinkedIn


In [6]:
def addDataToDB(data):
    conn = sqlite3.connect('labelled_data.db')
    cursor = conn.cursor()
    cursor.execute('''
                   CREATE TABLE IF NOT EXISTS labelled_job_postings(
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    job_posting_id TEXT,
                    title TEXT,
                    company TEXT,
                    location TEXT,
                    description TEXT,
                    job_site TEXT,
                    sponsorship_available TEXT
                   )
                   
                   '''
    )
    conn.commit()
    cursor.executemany('''
                        INSERT INTO labelled_job_postings(job_posting_id, title, company, location, description, job_site)
                        VALUES (?, ?, ?, ?, ?, ?)
                    ''', data.values.tolist()
    )
    conn.commit()
    conn.close()


In [10]:
def cleanData():
    df = pd.read_csv("linkedin_data.csv")
    df_cleaned = df.drop_duplicates(subset="job_posting_id", keep="first")
    df_cleaned = df.dropna()
    addDataToDB(df_cleaned)
    # df_cleaned.to_csv('linkedin_data_cleaned.csv', index=False)
cleanData()

In [11]:
def readDbData():
    conn = sqlite3.connect('labelled_data.db')
    cursor = conn.cursor()
    cursor.execute('''
                   SELECT count(*) FROM  labelled_job_postings
                   
                   '''
    )
    rowcount = cursor.fetchone()[0]
    print(rowcount)
    conn.close()

readDbData()

18916
